In [50]:
import wntr
import networkx as nx

In [51]:
import pickle

In [52]:
import numpy as np

In [53]:
Wdn = wntr.network.WaterNetworkModel('EXN.inp')

In [54]:
Wdn.options.time.duration = 1 * 3600
Sim = wntr.sim.EpanetSimulator(Wdn)
Results = Sim.run_sim()

In [55]:
Flow = Results.link['flowrate']

In [56]:
def build_directed_network(wdn, flow, t):
    # Build directed graphs 
    dgraph = nx.DiGraph()
    for name, link in wdn.links():
        # Determine the direction of edge through the flow rate, positive: as stored, negative: change direction
        linkflow = flow.loc[3600*t, name]
        if linkflow >= 0:
            start_node = link.start_node_name
            end_node = link.end_node_name
        else:
            start_node = link.end_node_name
            end_node = link.start_node_name
        # Build corresponding original network
        dgraph.add_node(start_node, pos = wdn.get_node(start_node).coordinates)
        dgraph.add_node(end_node, pos = wdn.get_node(end_node).coordinates)
        dgraph.add_edge(start_node, end_node, linkid = name)
    return dgraph 

In [57]:
T = 1
Dgraph = build_directed_network(Wdn, Flow, T)

In [58]:
# Save the directed graph to pickle 
FDgraph = open(r"%s.pickle" % ('EXN_Dgraph'), 'wb')
pickle.dump(Dgraph, FDgraph)
FDgraph.close()

In [59]:
# Obtain the shortest distance to each WTPs, if there is no path, the shortest distance will be 10000
def node_shortestdistance_wtp(dgraph, sources):
    node_list = list(dgraph.nodes)
    node_shortestdistance = {}
    for node in node_list:
        shortestdistance = 10000
        for s in sources:
            distance = 10000
            # If there is a path between the source and the node
            try:
                path = nx.algorithms.shortest_paths.generic.shortest_path(dgraph, source=s, target=node)
                distance = len(path)
            except:
                pass
            if distance < shortestdistance:
                shortestdistance = distance
        node_shortestdistance[node] = shortestdistance
    return node_shortestdistance         

In [60]:
Sources = Wdn.reservoir_name_list
Node_shortestdistance = node_shortestdistance_wtp(Dgraph, Sources)

In [61]:
FNode_shortestdistance = open(r'EXN_Node_shortestdistance.pickle', 'wb')
pickle.dump(Node_shortestdistance, FNode_shortestdistance)
FNode_shortestdistance.close()

In [62]:
Cycles = list(nx.simple_cycles(Dgraph))

In [63]:
Cycles

[['288', '316'], ['663', '649']]

In [46]:
def itrate_cyclecs(cycles, node_shortestdistance):
    # downsnodes to store all the nodes in the cycles
    alldownnodes = []
    # allleadnodes to store all the nodes that have the shortest distance to sources in each cycle
    allleadnodes = []
    for c in cycles:
        # Initialize the lead node as the first node in the cycle
        leadnode = c[0]
        shortestdistance = node_shortestdistance[c[0]]
        for node in c:
            distance = node_shortestdistance[node]
            # if there s
            if distance < shortestdistance:
                leadnode = node
        # If this lead node is not the downnodes of other cycles
        if leadnode not in alldownnodes:
            allleadnodes.append(leadnode)
            alldownnodes += c
            alldownnodes.remove(leadnode)
            alldownnodes = list(set(alldownnodes))
        else:
            alldownnodes += c
            alldownnodes = list(set(alldownnodes))
    return allleadnodes, alldownnodes

In [47]:
Allleadnodes, Cycledownnodes = itrate_cyclecs(Cycles, Node_shortestdistance)

In [48]:
FAllleadnodes = open(r'ky12_Cycle_leadnodes_9am.pickle', 'wb')
pickle.dump(Allleadnodes, FAllleadnodes)
FAllleadnodes.close()

In [49]:
FCycledownnodes = open(r'ky12_Cycle_downnodes_9am.pickle', 'wb')
pickle.dump(Cycledownnodes, FCycledownnodes)
FCycledownnodes.close()

### Withdraw links that to retain

In [15]:
Link_flow = dict(Flow.loc[T*3600])

In [37]:
Flow_threshold = np.percentile(list(Link_flow.values()), 5)

In [25]:
# Get the median diameter as the threshold
Pipelist = Wdn.pipe_name_list
Diameter = []
for P in Pipelist:
    Pobject = Wdn.get_link(P)
    Diameter.append(Pobject.diameter)
Diameter = np.array(Diameter)

0.152

In [33]:
np.percentile(Diameter, 90)

0.305

In [38]:
def links_remove(wdn, diameter_threshold, flow_threshold, link_flow):
    remove = []
    for name, linkobject in wdn.links():
        try:
            d = linkobject.diameter
            if (d <= diameter_threshold) and (link_flow[name] < flow_threshold):
                remove.append(name)
        except:
            pass
    return remove 

In [39]:
Diameter_threshold = 0.305
Remove = links_remove(Wdn, Diameter_threshold, Flow_threshold, Link_flow)

In [40]:
Flinks_remove = open(r'Dtown_links_remove_7am.pickle', 'wb')
pickle.dump(Remove, Flinks_remove)
Flinks_remove.close()